# Data Process to aggregate routines

This is guide to aggregate routines

---

In location.ipynb, we analyze location data and derive routines from location data.
Here, we are going to aggregate those routines to visualize routines.

Please, run location.ipynb first.

## Aggregate daily

aggregate for daily routine

DataFrame schema
- start_at
- end_at
- routine
- user_id


In [11]:
# Please change USER_ID
USER_ID = 'P3029'
LOCATION_DATASET_DIRECTORY = f'../csv/routines_raw/{USER_ID}-location.csv'
SLEEP_DATASET_DIRECTORY = f'../csv/routines_raw/{USER_ID}-sleep.csv'

In [12]:
import pandas as pd

sleep_df = pd.read_csv(SLEEP_DATASET_DIRECTORY)[['user_id', 'start_at', 'end_at', 'routine']]
sleep_df['start_at'] = pd.to_datetime(sleep_df['start_at'])
sleep_df['end_at'] = pd.to_datetime(sleep_df['end_at'])
sleep_df['weekday'] = sleep_df['start_at'].dt.dayofweek


sleep_df

,user_id,start_at,end_at,routine,weekday
0,P3029,2019-05-01 03:00:00+09:00,2019-05-01 08:15:00+09:00,SLEEP,2
1,P3029,2019-05-02 03:45:00+09:00,2019-05-02 08:15:00+09:00,SLEEP,3
2,P3029,2019-05-03 02:45:00+09:00,2019-05-03 08:45:00+09:00,SLEEP,4
3,P3029,2019-05-04 03:45:00+09:00,2019-05-04 09:15:00+09:00,SLEEP,5
4,P3029,2019-05-05 03:00:00+09:00,2019-05-05 07:30:00+09:00,SLEEP,6
5,P3029,2019-05-06 03:45:00+09:00,2019-05-06 12:00:00+09:00,SLEEP,0


In [13]:

timedelta = pd.to_timedelta('15min')
split_sleep=[]

for i in range(len(sleep_df)):
    sleep_df_row = sleep_df.iloc[i].tolist()
    starttime = sleep_df_row[1]
    while starttime+timedelta <= sleep_df_row[2]:
        newrow = [sleep_df_row[0], starttime, starttime+timedelta, sleep_df_row[3], sleep_df_row[4]]
        split_sleep.append(newrow)
        starttime+=timedelta

split_sleep_df = pd.DataFrame(split_sleep, columns=['user_id', 'start_at', 'end_at', 'routine', 'weekday'])

split_sleep_df
    

,user_id,start_at,end_at,routine,weekday
0,P3029,2019-05-01 03:00:00+09:00,2019-05-01 03:15:00+09:00,SLEEP,2
1,P3029,2019-05-01 03:15:00+09:00,2019-05-01 03:30:00+09:00,SLEEP,2
2,P3029,2019-05-01 03:30:00+09:00,2019-05-01 03:45:00+09:00,SLEEP,2
3,P3029,2019-05-01 03:45:00+09:00,2019-05-01 04:00:00+09:00,SLEEP,2
4,P3029,2019-05-01 04:00:00+09:00,2019-05-01 04:15:00+09:00,SLEEP,2
...,...,...,...,...,...
131,P3029,2019-05-06 10:45:00+09:00,2019-05-06 11:00:00+09:00,SLEEP,0
132,P3029,2019-05-06 11:00:00+09:00,2019-05-06 11:15:00+09:00,SLEEP,0
133,P3029,2019-05-06 11:15:00+09:00,2019-05-06 11:30:00+09:00,SLEEP,0
134,P3029,2019-05-06 11:30:00+09:00,2019-05-06 11:45:00+09:00,SLEEP,0


In [14]:
raw_df = pd.concat([
    pd.read_csv(LOCATION_DATASET_DIRECTORY)[['user_id', 'start_at', 'end_at', 'routine', 'weekday']],
    split_sleep_df
])
raw_df['start_at'] = pd.to_datetime(raw_df['start_at'])
raw_df['end_at'] = pd.to_datetime(raw_df['end_at'])
raw_df

,user_id,start_at,end_at,routine,weekday
0,P3029,2019-04-30 09:00:00+09:00,2019-04-30 09:15:00+09:00,CLASS,1
1,P3029,2019-04-30 09:45:00+09:00,2019-04-30 10:00:00+09:00,CLASS,1
2,P3029,2019-04-30 10:00:00+09:00,2019-04-30 10:15:00+09:00,CLASS,1
3,P3029,2019-04-30 10:15:00+09:00,2019-04-30 10:30:00+09:00,CLASS,1
4,P3029,2019-04-30 11:00:00+09:00,2019-04-30 11:15:00+09:00,CLASS,1
...,...,...,...,...,...
131,P3029,2019-05-06 10:45:00+09:00,2019-05-06 11:00:00+09:00,SLEEP,0
132,P3029,2019-05-06 11:00:00+09:00,2019-05-06 11:15:00+09:00,SLEEP,0
133,P3029,2019-05-06 11:15:00+09:00,2019-05-06 11:30:00+09:00,SLEEP,0
134,P3029,2019-05-06 11:30:00+09:00,2019-05-06 11:45:00+09:00,SLEEP,0


In [15]:
daily_df = raw_df.copy()
daily_df['end_at'] = daily_df["end_at"].replace(["00:00:00"], "23:59:59")
daily_df['start_at'] = daily_df['start_at'].dt.time
daily_df['end_at'] = daily_df['end_at'].dt.time
     

daily_df = daily_df.groupby(['user_id', 'start_at', 'end_at']) \
                   .agg(routine=('routine', lambda x: pd.Series.mode(x)[0])) \
                   .reset_index()


daily_df


,user_id,start_at,end_at,routine
0,P3029,00:00:00,00:15:00,INDOOR
1,P3029,00:15:00,00:30:00,INDOOR
2,P3029,00:30:00,00:45:00,INDOOR
3,P3029,00:45:00,01:00:00,STUDY
4,P3029,02:00:00,02:15:00,STUDY
...,...,...,...,...
75,P3029,21:30:00,21:45:00,STUDY
76,P3029,21:45:00,22:00:00,INDOOR
77,P3029,22:45:00,23:00:00,STUDY
78,P3029,23:15:00,23:30:00,STUDY


In [16]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   80 non-null     object
 1   start_at  80 non-null     object
 2   end_at    80 non-null     object
 3   routine   80 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


### Check with Daily Routine

In [18]:
import plotly.express as px

daily_fig_df = daily_df.copy()
daily_fig_df['start_at'] = pd.to_datetime(daily_df['start_at'], format='%H:%M:%S')
daily_fig_df['end_at'] = daily_fig_df['start_at'] + pd.Timedelta(minutes=15)
fig = px.timeline(
    daily_fig_df,
    x_start='start_at',
    x_end='end_at',
    y='user_id',
    color='routine',
    height=400,
    width=1200,
)
fig.update_xaxes(tickformat="%H:%M")

fig.show()

### Export daily routine

In [19]:
daily_df.to_csv(f'../csv/routines/{USER_ID}-daily.csv')

-----

## Aggregate weekly

In [20]:
weekly_df = raw_df.copy()
weekly_df['start_at'] = weekly_df['start_at'].dt.time
weekly_df['end_at'] = weekly_df['end_at'].dt.time

weekly_df = weekly_df.groupby(['user_id', 'start_at', 'end_at', 'weekday']) \
    .agg(routine=('routine', lambda x: pd.Series.mode(x)[0])) \
    .reset_index()

weekly_df

,user_id,start_at,end_at,weekday,routine
0,P3029,00:00:00,00:15:00,4,INDOOR
1,P3029,00:15:00,00:30:00,2,INDOOR
2,P3029,00:15:00,00:30:00,4,INDOOR
3,P3029,00:30:00,00:45:00,4,INDOOR
4,P3029,00:45:00,01:00:00,4,STUDY
...,...,...,...,...,...
223,P3029,21:45:00,22:00:00,3,STUDY
224,P3029,21:45:00,22:00:00,4,INDOOR
225,P3029,22:45:00,23:00:00,3,STUDY
226,P3029,23:15:00,23:30:00,3,STUDY


### Check with weekly routine

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

weekly_fig_df = weekly_df[weekly_df['routine'] != 'INDOOR']
weekly_fig_df['start_at'] = pd.to_datetime(weekly_df['start_at'], format='%H:%M:%S')
weekly_fig_df['end_at'] = weekly_fig_df['start_at'] + pd.Timedelta(minutes=15)

fig = make_subplots(rows=7, cols=1, shared_xaxes=True, vertical_spacing=0.02)
timeline_figs = []
for i in range(7):
    timeline_figs.append(
        px.timeline(weekly_fig_df[weekly_fig_df["weekday"] == i], x_start="start_at", x_end="end_at", y="user_id", color="routine")
    )

for i in range(7):
    f = timeline_figs[i]
    show_legend = True if i == 0 else False
    for j in range(len(f.data)):
        fig.add_trace(go.Bar(f.data[j], showlegend=show_legend), row=(i + 1), col=1)

fig.update_yaxes(title="Mon", row=1, col=1)
fig.update_yaxes(title="Tue", row=2, col=1)
fig.update_yaxes(title="Wed", row=3, col=1)
fig.update_yaxes(title="Thr", row=4, col=1)
fig.update_yaxes(title="Fri", row=5, col=1)
fig.update_yaxes(title="Sat", row=6, col=1)
fig.update_yaxes(title="Sun", row=7, col=1)

fig.update_xaxes(tickformat="%H:%M")
fig.update_layout(title="Weekly Routine")

fig.update_xaxes(type="date")

fig.update_layout(width=700, height=600)

C:\Users\user\AppData\Local\Temp\ipykernel_15544\1798017569.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_15544\1798017569.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Export weekly routine

In [23]:
weekly_df.to_csv(f'../csv/routines/{USER_ID}-weekly.csv', index=False)